# Relational data and databases

Original source: http://www.datasciencecourse.org/notes/relational_data/

In [21]:
import pandas as pd

df = pd.DataFrame([(1, 'Kolter', 'Zico'), 
                   (2, 'Xi', 'Edgar'),
                   (3, 'Lee', 'Mark'), 
                   (4, 'Mani', 'Shouvik'),
                   (5, 'Gates', 'Bill'),
                   (6, 'Musk', 'Elon')], 
                  columns=["id", "last_name", "first_name"])
df

,id,last_name,first_name
0,1,Kolter,Zico
1,2,Xi,Edgar
2,3,Lee,Mark
3,4,Mani,Shouvik
4,5,Gates,Bill
5,6,Musk,Elon


The first column in the displayed data frame is the "index" of each row.  Unfortunately the terminology here clashes with database terminology, but "index" for Pandas actually means something more like "primary key" in a database table (though with the exception that it _is_ possible to have duplicate entries, though I'd always recommend avoiding this in Pandas).  That is, an index (if done right, without duplicate indices) is a identifier for each row in the database.  We can set the index to one of the existing columns using the `.set_index()` call.

In [24]:
df.set_index("id")

,last_name,first_name
id,,
1,Kolter,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon


In [ ]:
df

But you need to be very careful about one thing here.  By default, most Pandas operations, like `.set_index()` and many others, and _not_ done in place.  That is, while the `df.set_index("id")` call above _returns_ a copy of the `df` dataframe with the index set to the `id` column (remember that Jupyter notebook displays the return value of the last line in a cell), the original `df` object is actually unchanged here:

In [7]:
df

,id,last_name,first_name
0,1,Kolter,Zico
1,2,Xi,Edgar
2,3,Lee,Mark
3,4,Mani,Shouvik
4,5,Gates,Bill
5,6,Musk,Elon


If we want to actually change the `df` object itself, you need to use the `inplace=True` flag for these functions (or assign the original object to the result of a function, but this isn't as clean):

In [36]:
df.set_index("id", inplace=True)

There are also some peculiarities that you'll get used to with Pandas, such as the fact that if you select a single column or row from a Pandas DataFrame, you don't get a single-column or single-row DataFrame, but an alternative Pandas type called a `Series`, which is kind of like a 1D data frame (but it has some different indexing rules).  There are reasons behind all these design decisions, even if they can seem a bit arbitrary at first.

In [26]:
df[['first_name']]

,first_name
0,Zico
1,Edgar
2,Mark
3,Shouvik
4,Bill
5,Elon


We won't go into too much detail about Pandas here, but you should familiarize yourself with some of the basic strategies for accessing elements in a Pandas DataFrame.  There are actually a few different ways to do this, but I usually advocate for using the `.loc` or `.iloc` properties when you want to access or set individual elements (other strategies often lead to confusion when accessing by different types of indices, etc.

### Common Pandas data access

Let's consider a few of the common ways to access or set data in a Pandas DataFrame.  You can access individual elements using the `.loc[row, column]` notation, where `row` denotes the index you are searching for and `column` denotes the column name.  For example, to access the last name of person with ID 1 we would execute:

In [29]:
df.tail(10)

,id,last_name,first_name
0,1,Kolter,Zico
1,2,Xi,Edgar
2,3,Lee,Mark
3,4,Mani,Shouvik
4,5,Gates,Bill
5,6,Musk,Elon


In [30]:
df.loc[1, "last_name"]

'Xi'

If we want to access _all_ last names, (or all elements in a particular row), we use the `:` wildcard.  For example

In [32]:
df.loc[:, :]

,id,last_name,first_name
0,1,Kolter,Zico
1,2,Xi,Edgar
2,3,Lee,Mark
3,4,Mani,Shouvik
4,5,Gates,Bill
5,6,Musk,Elon


Notice the output here looks a bit different than the nicely-formed typical Pandas output, which stems from the fact (mentioned previously), that this returne object is actually not a Pandas DataFrame, but a Pandas Series, which (apparently) no one wants to write a nice display routine for.  If we do want to get a DataFrame with just this one column, we can get a "list" of columns (with just one elements)

In [6]:
df.loc[:, ["last_name"]]

,last_name
id,
1,Kolter
2,Xi
3,Lee
4,Mani
5,Gates
6,Musk


which now gives us the desired 1D data frame.  We can do a similar thing with row indexes.

In [7]:
df.loc[[1,2],:]

,last_name,first_name
id,,
1,Kolter,Zico
2,Xi,Edgar


which will select only a subject of the allowable rows.

We can additionally use `.loc` to change the content of existing entries:

In [8]:
df.loc[1,"last_name"] = "Kilter"
df

,last_name,first_name
id,,
1,Kilter,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon


We can even add additional rows/columns that don't exist.

In [9]:
df.loc[7,:] = ('Moore', 'Andrew')
df

,last_name,first_name
id,,
1,Kilter,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon
7,Moore,Andrew


Finally, remember that `.loc` always indexes based upon the "index" (i.e., effectively primary key) of the data frame along with the column name.  If you want to instead access based upon positional index (i.e., using 0-indexed counters for both the rows and columns), you can use the `.iloc` property

In [41]:
df
df.iloc[4,1]

'Bill'

### SQLite

Unlike Pandas, SQLite actually is a full-featured database, but unlike most production databases, it does not use a client/server model.  Databases are instead stored direclty on disk and accessed just via the library.  This has the advantage of being very simple, with no server to configure and run, but for large applications it is typically insufficient: because files are not very good at concurrent access (that is, many different processes/threads cannot simultaneously read and write from a single file), the system is not ideal for very large databases where multiple threads need to be constant readings from and writing to the database.  Note that SQLite _does_ have some limited forms of concurrency in this respect, but nothing sophisticated when compared to larger scale databases.  If you do want to investigate a more "production strength" client/server database, I'd highly recommend looking into PostreSQL.

SQLite, as the name suggests, uses the SQL (structured query language) language for interacting with the database; note both "Sequel" and "Ess Queue Ell" are acceptable prononciations of SQL, but I personally learned it as "Sequel", so would be completely unable to do anything else.

Interacting with SQLite (or any other SQL-based database) from Python is not ideal, because you typically use Python code to generate SQL expressions as strings, then execute them, which is not the most beautiful coding paradigm.  For simple databases, though, it usually suffices to get the job done.

Let's look at how to create a simple database with the "Person" and "Grades" tables that we had considered earlier.

In [48]:
import sqlite3
conn = sqlite3.connect("database.db")
cursor = conn.cursor()

### when you are done, call conn.close()

This code imports the library, creates a connection to the "database.db" file (it will create it if it does not already exist), and then creates a "cursor" into the database.  The notion of cursor is common to a lot of database libraries, but essentially a cursor is an object that allows us to interact with the database.  If we want to create the Person and Grades tables we saw above (to keep things simple, and later to illustrate joins, we'll use the first version of the Grages table, with no associative table), we would use the following syntax.

In [49]:
cursor.execute("""
CREATE TABLE person (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    first_name TEXT
);""")

cursor.execute("""
CREATE TABLE grades (
    person_id INTEGER PRIMARY KEY,
    hw1_grade INTEGER,
    hw2_grade INTEGER
);""")

conn.commit()

Let's insert some data into these tables.  The syntax for this operation, hopefully fairly straightforward is given by the following:

In [50]:
cursor.execute("INSERT INTO person VALUES (1, 'Kolter', 'Zico');")
cursor.execute("INSERT INTO person VALUES (2, 'Xi', 'Edgar');")
cursor.execute("INSERT INTO person VALUES (3, 'Lee', 'Mark');")
cursor.execute("INSERT INTO person VALUES (4, 'Mani', 'Shouvik');")
cursor.execute("INSERT INTO person VALUES (5, 'Gates', 'Bill');")
cursor.execute("INSERT INTO person VALUES (6, 'Musk', 'Elon');")

cursor.execute("INSERT INTO grades VALUES (5, 85, 95);")
cursor.execute("INSERT INTO grades VALUES (6, 80, 60);")
cursor.execute("INSERT INTO grades VALUES (100, 100, 100);")

[Note that I'm an additional row to the `grades` tables with `person_id=100` for illustration purposes later when we talk about joins.  Notice that this actually would violate the consistency of the data if we had required `grades.person_id` to be a true foreign key, but since we didn't bother, it works fine].  If we want to see what has been added to the database, we can do this the "SQLite Python" way, which involves running a query and then iterating over the rows in a result returned by a `cursor.execute()` result, as so:

In [51]:
for row in cursor.execute("SELECT * FROM person;"):
    print(row)

(1, 'Kolter', 'Zico')
(2, 'Xi', 'Edgar')
(3, 'Lee', 'Mark')
(4, 'Mani', 'Shouvik')
(5, 'Gates', 'Bill')
(6, 'Musk', 'Elon')


Alternatively, it can be handy to dump the results of a query directly into a Pandas DataFrame.  Fortunately, Pandas provides a nice call for doing this, the `pd.read_sql_query()` function, with takes the database connection and an optional argument to set the index of the Pandas dataframe to be one of the columns.

In [15]:
pd.read_sql_query("SELECT * from person;", conn, index_col="id")

,last_name,first_name
id,,
1,Kolter,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon


The `SELECT` statement is probably the SQL command you'll use most in data science: it is used to query data from th database.  We won't here go into a full description of the `SELECT` statement, except to say that the most common syntax is something like:

`<columns>` here will be a comma-separated list of all the columns to select, or the wildcard `*` to denote all columns.  `<tables>` is a comma-separated list of tables.  And `<conditions>` is a list of conditions, typically separated by `AND` if there are multiple conditions, that specify what subset to return.  For example, let's see how to select all the last name (and id) from the persons table with `id > 2`

In [76]:
pd.read_sql_query("SELECT id,last_name FROM person WHERE id > 2;", conn, index_col="id")

,last_name
id,
3,Lee
4,Mani
5,Gates
6,Musk


**Note**: SQL commands are case insensitive, so using all caps for the statements is purely a convention.

Lastly, we can also delete values from tables using the `DELETE FROM` SQL command, using a similar `WHERE` clause as in the `SELECT` command.

In [16]:
cursor.execute("INSERT INTO person VALUES (7, 'Moore', 'Andrew');")
pd.read_sql_query("SELECT * from person;", conn, index_col="id")

,last_name,first_name
id,,
1,Kolter,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon
7,Moore,Andrew


In [71]:
cursor.execute("DELETE FROM person where id = 7;")
pd.read_sql_query("SELECT * from person;", conn, index_col="id")            

,last_name,first_name
id,,
1,Kolter,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon


## Joins

The last topic we'll consider here are joins between tables.  Briefly, join operations multiple multiple tables into a single relation, matching between attributes in the two tables.  There are four types of joins, though only the first two are used much in practice:

1. Inner
2. Left
3. Right
4. Outer

You join two tables _on columns_ from each table, where these columns specify how to match the rows between the two columns.  This should become more clear with a few examples.  In the examples that follow, we're going to consider our Person and Grades tables, that we just created above, and we're join to join the tables on the `person.id` and `grades.person_id` columns.

### Inner joins

If you don't know what type of join you want, you probably want an inner join.  This does the "obvious" thing, of only returning those rows where the two columns in each table have matching values, and it appends the rows together for each of these matching rows.  In our case, an inner join between the Person and Grades table would return the following:

| ID | Last Name | First Name | HW1 Grade | HW2 Grade |
| :---: | :---: | :---: | :---: | :---: |
| 5 | Gates | Bill | 85 | 95 |
| 6 | Musk | Elon | 80 | 60 |

Let's see how this can be done programatically, first in Pandas and then in SQL.  In Pandas, you should do joins with the `.merge()` command: there is an alternative `.join()` command, but this always assumes you want to join on the index column for one of the data frames, and _not_ the index frame for another, and overall is just a special case of `.merge()` so I prefer to learn the generic call.

In [17]:
df_person = pd.read_sql_query("SELECT * FROM person", conn)
df_grades = pd.read_sql_query("SELECT * FROM grades", conn)
df_person.merge(df_grades, how="inner", left_on = "id", right_on="person_id")

,id,last_name,first_name,person_id,hw1_grade,hw2_grade
0,5,Gates,Bill,5,85,95
1,6,Musk,Elon,6,80,60


Hopefully the syntax of `.merge()` looks straightforward: you call the method on the "left" data frame, and pass the "right" data frame as the first argument.  The `how` parameter specifies the type of join (here inner), and the `left_on` and `right_on` arguments specify the column name that you want to join on for the left and right data frames respectively.  If you alternatively want to join on the index for the left or right data frame, you specify the `left_index` or `right_index` parameters as so:

In [18]:
df_person.set_index("id", inplace=True)
df_grades.set_index("person_id", inplace=True)

In [19]:
df_person.merge(df_grades, how="inner", left_index=True, right_index=True)

,last_name,first_name,hw1_grade,hw2_grade
5,Gates,Bill,85,95
6,Musk,Elon,80,60


This call is maybe a bit cleaner, as it doesn't include the (duplicate) columns you match on as in the previous example, but this is a minor difference.

In SQL, an inner join is specified by the `WHERE` clause in a `SELECT` statement.

In [20]:
pd.read_sql_query("SELECT * FROM person, grades WHERE person.id = grades.person_id" , conn)

,id,last_name,first_name,person_id,hw1_grade,hw2_grade
0,5,Gates,Bill,5,85,95
1,6,Musk,Elon,6,80,60


Exactly the same behavior as our Pandas join, but the advantage of course is that for very complex joins and large database queries, a true database will be faster at performing queries like this than the Pandas code.

### Left joins

Whereas an inner join only kept those rows with corresponding entires in both tables, a left join will keep _all_ the items in the left table, and add in the attribution from the right table (filling with NaNs if no match exists in the right table).  Any row value that occurs in the right table but not the left table is discarded.

For the rest of this section, we'll simply write the Pandas code to perform the relevant join, then show the output it produces, rather than explicitly write the table that results.

In [94]:
df_person = pd.read_sql_query("SELECT * FROM person", conn)
df_grades = pd.read_sql_query("SELECT * FROM grades", conn)
df_person.merge(df_grades, how="left", left_on = "id", right_on="person_id")

,id,last_name,first_name,person_id,hw1_grade,hw2_grade
0,1,Kolter,Zico,NaN,NaN,NaN
1,2,Xi,Edgar,NaN,NaN,NaN
2,3,Lee,Mark,NaN,NaN,NaN
3,4,Mani,Shouvik,NaN,NaN,NaN
4,5,Gates,Bill,5.0,85.0,95.0
5,6,Musk,Elon,6.0,80.0,60.0


Note for the two students, we properly fill in the grades, but for everyone else will fill in NaN values for the grades, because the person did not have any associated grades in the table.

SQL syntax for left join use the `LEFT JOIN` statement, as follows:

In [91]:
pd.read_sql_query("SELECT * FROM person LEFT JOIN grades ON person.id = grades.person_id" , conn)

,id,last_name,first_name,person_id,hw1_grade,hw2_grade
0,1,Kolter,Zico,NaN,NaN,NaN
1,2,Xi,Edgar,NaN,NaN,NaN
2,3,Lee,Mark,NaN,NaN,NaN
3,4,Mani,Shouvik,NaN,NaN,NaN
4,5,Gates,Bill,5.0,85.0,95.0
5,6,Musk,Elon,6.0,80.0,60.0


### Right joins

A right join does what you might expect, the converse of the left join, where all the rows in the right matrix are kept.  While SQLite has no syntax for right joins (you can achieve the same results by simply reversing the order of the two tables and doing a left join), Pandas does have built-in syntax for the right join

In [95]:
df_person.merge(df_grades, how="right", left_on = "id", right_on="person_id")

,id,last_name,first_name,person_id,hw1_grade,hw2_grade
0,5.0,Gates,Bill,5,85,95
1,6.0,Musk,Elon,6,80,60
2,NaN,NaN,NaN,100,100,100


### Outer joins

Finally, outer joins (also called a cross product) do what you may expect, and keep all rows that occur in either table, so essentially take the union of the left and right joins.  Again, SQLite has no syntax for it (you can achieve the same thing via a `UNION` statement, but we won't cover it), but Pandas has the function.

In [96]:
df_person.merge(df_grades, how="outer", left_on = "id", right_on="person_id")

,id,last_name,first_name,person_id,hw1_grade,hw2_grade
0,1.0,Kolter,Zico,NaN,NaN,NaN
1,2.0,Xi,Edgar,NaN,NaN,NaN
2,3.0,Lee,Mark,NaN,NaN,NaN
3,4.0,Mani,Shouvik,NaN,NaN,NaN
4,5.0,Gates,Bill,5.0,85.0,95.0
5,6.0,Musk,Elon,6.0,80.0,60.0
6,NaN,NaN,NaN,100.0,100.0,100.0


While you will probably use inner joins the vast majority of the time (and left joins the remainder), it is helpful to understand the different types of join operations from a conceptual standpoint, and how they all fit together.

## References

- [Pandas](https://pandas.pydata.org/)
- [SQLite](https://www.sqlite.org)
- [SQLite SQL syntax](https://www.sqlite.org/lang.html)
- [Python sqlite3 library](https://docs.python.org/3/library/sqlite3.html)